# Process Data

In [124]:
import pandas as pd
import re
import string
from nltk.tokenize import RegexpTokenizer


from gensim.utils import tokenize

import nltk

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from nltk import sent_tokenize, word_tokenize
from textblob import TextBlob

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [125]:
# Load Data
df_test_sample = pd.read_pickle('..//data//processed//df_test_sample.pkl')
df_train_sample = pd.read_pickle('..//data//processed//df_train_sample.pkl')
df_comments_sample = pd.read_pickle('..//data//processed//df_comments_sample.pkl')
df_articles_sample = pd.read_pickle('..//data//processed//df_articles_sample.pkl')

In [126]:
sentence = df_test_sample.iloc[6]['abstract']
list(tokenize(sentence))

['A',
 'Canadian',
 's',
 'gruesome',
 'account',
 'as',
 'an',
 'Islamic',
 'State',
 'executioner',
 'in',
 'Syria',
 'which',
 'was',
 'the',
 'subject',
 'of',
 'the',
 'Caliphate',
 'podcast',
 'by',
 'The',
 'New',
 'York',
 'Times',
 'was',
 'fabricated',
 'officials',
 'say',
 'A',
 'Times',
 'review',
 'found',
 'no',
 'corroboration',
 'of',
 'his',
 'claim',
 'to',
 'have',
 'committed',
 'atrocities']

### split into words and handling punctuation

In [127]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [128]:
text = df_test_sample['abstract'].str.replace('[^a-zA-Z]',' ')
text


C:\Users\Mike\AppData\Local\Temp/ipykernel_12892/1415527003.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  text = df_test_sample['abstract'].str.replace('[^a-zA-Z]',' ')


1508    How four traumatic years turned Georgia into a...
3873    On Jan     the vice president will preside as ...
2875    One day  we ll look back on this year and bawl...
2202    Have you seen more skateboarding parks in your...
305     New York City had been holding off a second wa...
                              ...                        
1859    The  Chanel of streetwear  has a new owner  an...
2432    The ownership of Taylor Swift s catalog return...
992     Teenage comments in response to our recent wri...
1116    Between a third and a half of all eligible vot...
1369    Donald Trump has made America exceptional all ...
Name: abstract, Length: 799, dtype: object

# Normalization
remove the noise

In [129]:
# Normalize
def normalize_col(df, column, column_new):
    #  https://towardsdatascience.com/text-cleaning-methods-
    #  for-natural-language-processing-f2fc1796e8c7
    df[column_new] = df[column].str.lower()
    df[column_new] = df[column].apply\
        (lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|\
        (\w+:\/\/\S+)|^rt|http.+?", "", elem))  

    df[column_new] = df[column].apply\
        (lambda elem: re.sub(r"\d+", "", elem))
    return df

In [130]:
data_clean = normalize_col(df_test_sample, 'abstract', 'abstract_nrml')
data_clean[['abstract','abstract_nrml']]

,abstract,abstract_nrml
1508,How four traumatic years turned Georgia into a...,How four traumatic years turned Georgia into a...
3873,"On Jan. 6, the vice president will preside as ...","On Jan. , the vice president will preside as C..."
2875,"One day, we’ll look back on this year and bawl...","One day, we’ll look back on this year and bawl..."
2202,Have you seen more skateboarding parks in your...,Have you seen more skateboarding parks in your...
305,New York City had been holding off a second wa...,New York City had been holding off a second wa...
...,...,...
1859,"The “Chanel of streetwear” has a new owner, an...","The “Chanel of streetwear” has a new owner, an..."
2432,The ownership of Taylor Swift’s catalog return...,The ownership of Taylor Swift’s catalog return...
992,Teenage comments in response to our recent wri...,Teenage comments in response to our recent wri...
1116,Between a third and a half of all eligible vot...,Between a third and a half of all eligible vot...


# Stopwords

In [136]:
# What are the stopwords
stop[0:5]

['i', 'me', 'my', 'myself', 'we']

In [137]:
def remove_stopwords(df, column, column_new):
    df[column_new] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return df

In [141]:
df_test_sample = remove_stopwords(df_test_sample, 'abstract_nrml','abstract_nrml_stp')

In [142]:
df_test_sample[['abstract','abstract_nrml','abstract_nrml_stp']]

,abstract,abstract_nrml,abstract_nrml_stp
1508,How four traumatic years turned Georgia into a...,How four traumatic years turned Georgia into a...,How four traumatic years turned Georgia swing ...
3873,"On Jan. 6, the vice president will preside as ...","On Jan. , the vice president will preside as C...","On Jan. , vice president preside Congress coun..."
2875,"One day, we’ll look back on this year and bawl...","One day, we’ll look back on this year and bawl...","One day, we’ll look back year bawl. But also r..."
2202,Have you seen more skateboarding parks in your...,Have you seen more skateboarding parks in your...,Have seen skateboarding parks area? Do think s...
305,New York City had been holding off a second wa...,New York City had been holding off a second wa...,"New York City holding second wave, uptick case..."
...,...,...,...
1859,"The “Chanel of streetwear” has a new owner, an...","The “Chanel of streetwear” has a new owner, an...","The “Chanel streetwear” new owner, tough balan..."
2432,The ownership of Taylor Swift’s catalog return...,The ownership of Taylor Swift’s catalog return...,The ownership Taylor Swift’s catalog returned ...
992,Teenage comments in response to our recent wri...,Teenage comments in response to our recent wri...,Teenage comments response recent writing promp...
1116,Between a third and a half of all eligible vot...,Between a third and a half of all eligible vot...,Between third half eligible voters typically s...


# Tokenize

# Lemmatization
Make words in root form, morphological analysis

In [146]:
def wrd_lemmatize(text):
    lemmaed = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lemmaed
    

In [147]:
df_test_sample.columns

Index(['newsdesk', 'section', 'subsection', 'material', 'headline', 'abstract',
       'keywords', 'word_count', 'pub_date', 'is_popular', 'uniqueID',
       'abstract_nrml', 'abstact_nrml_stp', 'abstract_nrml_stp',
       'abstract_nrml_stp_tokn'],
      dtype='object')

In [ ]:
df_test_sample['no']

In [148]:
df_test_sample['text_tokens_lemmaed'] = df_test_sample['abstract_nrml_stp'] \
                    .apply(lambda x: wrd_lemmatize(x))

In [149]:
df_test_sample['text_tokens_lemmaed'] 

1508    [H, o, w,  , f, o, u, r,  , t, r, a, u, m, a, ...
3873    [O, n,  , J, a, n, .,  , ,,  , v, i, c, e,  , ...
2875    [O, n, e,  , d, a, y, ,,  , w, e, ’, l, l,  , ...
2202    [H, a, v, e,  , s, e, e, n,  , s, k, a, t, e, ...
305     [N, e, w,  , Y, o, r, k,  , C, i, t, y,  , h, ...
                              ...                        
1859    [T, h, e,  , “, C, h, a, n, e, l,  , s, t, r, ...
2432    [T, h, e,  , o, w, n, e, r, s, h, i, p,  , T, ...
992     [T, e, e, n, a, g, e,  , c, o, m, m, e, n, t, ...
1116    [B, e, t, w, e, e, n,  , t, h, i, r, d,  , h, ...
1369    [D, o, n, a, l, d,  , T, r, u, m, p,  , m, a, ...
Name: text_tokens_lemmaed, Length: 799, dtype: object